In [2]:
pip install pubchempy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [77]:
import pubchempy as pcp
import pickle
import pandas as pd
import numpy as np

In [88]:
#HSDN disease-symptom association
disease_symptom = pd.read_csv('/Users/pengc/Documents/DS5500/dataset/disease-symptom.csv')
disease = disease_symptom['MeSH Disease Term'].unique()

In [89]:
#CTD drug-disease association preprocessing
df = pd.read_csv('/Users/pengc/Documents/DS5500/dataset/CTD_chemicals_diseases.csv', skiprows=27, usecols = ['# ChemicalName','ChemicalID','DiseaseName','DiseaseID','DirectEvidence'])
df.rename(columns = {'# ChemicalName':'ChemicalName'}, inplace = True)
df = df[df['DirectEvidence']=='therapeutic']
df = df[~df['ChemicalName'].str.contains('drug combination')]
df = df.dropna(subset=['ChemicalName','ChemicalID','DiseaseName','DiseaseID'])
df = df[df['DiseaseName'].isin(disease)]#match with HSDN
#print(len(df))
df.head()

33323


,ChemicalName,ChemicalID,DiseaseName,DiseaseID,DirectEvidence
1,06-Paris-LA-66 protocol,C046983,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic
163,"10,11-dihydro-10-hydroxycarbamazepine",C039775,Epilepsy,MESH:D004827,therapeutic
1102,10-hydroxycamptothecin,C028098,Huntington Disease,MESH:D006816,therapeutic
1114,10-hydroxycamptothecin,C028098,Liver Neoplasms,MESH:D008113,therapeutic
1365,"10-methoxy-2,2-dimethyl-2,6-dihydropyrano(3,2-...",C554291,Leukemia,MESH:D007938,therapeutic


In [91]:
chemName = df['ChemicalName'].unique()
DiseaseName = df['DiseaseName'].unique()
#len(chemName), len(DiseaseName)

(6473, 2169)

In [101]:
chem_compound = {}
for i, chem in enumerate(chemName):
  res = pcp.get_compounds(chem, 'name')
  if len(res)==1:
    chem_compound[chem] = res[0]

PubChemHTTPError: 'PUGREST.ServerBusy'

In [50]:
with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(chem_compound, f)

In [51]:
with open('saved_dictionary.pkl', 'rb') as f:
        drug_cid = pickle.load(f)

In [97]:
#reference: https://github.com/mcs07/PubChemPy/blob/master/examples/Chemical%20fingerprints%20and%20similarity.ipynb
def tanimoto(compound1, compound2):
    fp1 = int(compound1.fingerprint, 16)
    fp2 = int(compound2.fingerprint, 16)
    fp1_count = bin(fp1).count('1')
    fp2_count = bin(fp2).count('1')
    both_count = bin(fp1 & fp2).count('1')
    return float(both_count) / (fp1_count + fp2_count - both_count)

In [113]:
compound = list(chem_compound.values())
tanimoto(compound[0],compound[1])

0.4595744680851064

In [112]:
chem_compound

{'10,11-dihydro-10-hydroxycarbamazepine': Compound(114709),
 '10-propargyl-10-deazaaminopterin': Compound(148121),
 '1,10-diaminodecane': Compound(1317),
 '1-(1,3-benzodioxol-5-ylcarbonyl)piperidine': Compound(1370),
 '1,1-dimethylbutyl-1-deoxy-Delta(9)-THC': Compound(6918505),
 '11-keto-boswellic acid': Compound(23675050),
 '1-(2-(1-adamantyl)ethyl)-1-pentyl-3-(3-(4-pyridyl)propyl)urea': Compound(10158562),
 '1-(2-(3-(4-methoxyphenyl)propoxy)-4-methoxyphenylethyl)-1H-imidazole': Compound(104956),
 '1,2,3,4-tetrahydroisoquinoline': Compound(7046),
 '1-(2,3-dihydro-1,4-benzodioxin-5-yl)-4-((5-(4-fluorophenyl)-3-pyridinyl)methyl)piperazine': Compound(9887537),
 '1,25-dihydroxy-16-ene-vitamin D3': Compound(6439178),
 '1,25-dihydroxy-24-oxo-16-ene-vitamin D3': Compound(9954323),
 '1,25-dihydroxyergocalciferol': Compound(9547243),
 '1,25-dihydroxyvitamin D': Compound(5280453),
 "1,2-bis(2-aminophenoxy)ethane N,N,N',N'-tetraacetic acid acetoxymethyl ester": Compound(3034747),
 '1-(2-chloroet